In [ ]:
import os
from datetime import datetime
import random

import numpy as np
from math import *

import tensorflow as tf

import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import KFold

# Tensorflow

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)
tf.test.is_built_with_cuda()

# Case

In [3]:
folders = ['Rough','UTS','Elon']
index = 1
CASE = folders[index]

In [4]:
from prepross import importForK
x,y = importForK(index)

# Network

In [ ]:
# Hyperparameters
lossfn = [tf.keras.losses.MeanSquaredError(),tf.keras.losses.MeanAbsolutePercentageError(),tf.keras.losses.MeanAbsoluteError()]
lossfn = lossfn[index]
optimizer = [tf.keras.optimizers.Adam(learning_rate=i,beta_1=0.09,beta_2=0.9) for i in [5e-2,1e-2,5e-3]]
optimizer = optimizer[index]
metrics = ['mean_squared_error','mean_absolute_error',tf.keras.losses.MeanAbsolutePercentageError()]

In [ ]:
#Give a random seed
tf.random.set_seed(random.randint(1,1000))
from networks import create_model,compile_and_fit

# K-Fold cross validation

In [ ]:
mArch = []
for l2 in range(3,11):
    for l3 in range(2,73):
        if l2*(len(x.columns)+l3) + l3 < 100 and l2*(len(x.columns)+l3) + l3 > 50:
            mArch.append([[l2,'tanh',1],[l3,'linear',1]])
            mArch.append([[l2,'tanh',1],[l3,'relu',1],1])

In [ ]:
def kfoldTrain(xd,yd,lossfn,mArch,n_split,modelName):
  error = {}
  error['MAE'] = 0
  error["MSE"] = 0
  error['MRE'] = 0
  i=1
  for train_index,test_index in KFold(n_split).split(xd):
    x_train,x_test=xd.values[train_index],xd.values[test_index]
    y_train,y_test=yd.values[train_index],yd.values[test_index]
    
    model = create_model(mArch,modelName+"_v"+str(i),len(x.columns))
    # Print Model Summary
    model.summary()

    # Compile And Fit
    history = compile_and_fit(CASE,model,optimizer,lossfn,metrics,
                            modelName+'_v'+str(i),x_train,y_train,
                            verbose = 0,batch_size=3,MAX_EPOCHS=300,
                            patience = 50,record = True)
 
    error['MSE'] += model.evaluate(x_test,y_test)[0]
    error['MAE'] += model.evaluate(x_test,y_test)[1]
    error['MRE'] += model.evaluate(x_test,y_test)[2]
    i+=1
  return error

In [ ]:
performance = {}
for ele in mArch:
    print(ele)
    # Create a basic model Architecture
    modelArchitecture = [ele[0],ele[1],[1,'linear',1]]
    modelName = ''
    for u,fn,b in modelArchitecture:
        modelName += str(u)+fn+str(b)+'_'
   
    performance[modelName] = kfoldTrain(x,y,lossfn,modelArchitecture,10,modelName)
    # print(val_performance[modelName])
    # print(performance[modelName])